# Проект: Retrieval-система по статьям с arXiv


## Постановка задачи

В проекте вам предстоит построить retrieval-систему и оценить её качество. У вас будет датасет с аннотациями статей с портала arXiv.org и тестовый набор запросов и правильных ответов к ним. Также выполнить профилирование частей модели.

## Критерии успеха
Качество поисковой выдачи (Mean Reciprocal Rank): **MRR@5 > 0.91**.

Проект считается выполненным при .

<div class="alert alert-secondary" style="background-color:#D9EEE1;color:black;">

## Описание данных
Дан архив с датасетом.

В нём есть метаданные статей `arxiv-metadata-s.json` и набор тестовых запросов для оценки качества поиска `test_sample.csv`.
- `test_sample.csv` имеет структуру:
    - `query` — запрос к поисковой системе,
    - `id` — ID статьи, отвечающей на запрос query (один запрос — один верный ответ),
    - `abstract` — аннотация статьи id.
- arxiv-metadata-s.json имеет структуру:
    - `id` — ID статьи,
    - `abstract` — аннотация статьи,
    - `title` — название статьи.

## Инициализация

In [1]:
ARXIV_METADATA_PATH = "arxiv-metadata-s.json"
TEST_SAMPLE_PATH = "test_sample.csv"
RANDOM_STATE = 42

In [2]:
# 👷🚧🚧🚧🚧🚧 просто для дебага без GPU - переключить в конце в False:
JUST_DEBUG_WO_GPU = False
DEBUG_MAX_ELEMENTS = 10

### Импорт библиотек

In [3]:
import pandas as pd
import json
from pandas import json_normalize

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List, Optional

import faiss
import numpy as np

from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

import torch
from torch import Tensor
import torch.nn.functional as F
from langchain_community.document_loaders import JSONLoader

/home/ubuntu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Исследовательский анализ (EDA)

Загрузим и зучим данные датасета.

База стататей для RAG:

In [4]:
with open(ARXIV_METADATA_PATH, "r", encoding="utf-8") as f:
    arxiv_metadata = json.load(f)

In [5]:
len(arxiv_metadata)

98213

In [6]:
arxiv_metadata[0]

{'id': '0704.0038',
 'submitter': 'Maxim A. Yurkin',
 'authors': 'Maxim A. Yurkin, Alfons G. Hoekstra',
 'title': 'The discrete dipole approximation: an overview and recent developments',
 'comments': '36 pages, 1 figure; added several corrections according to the\n  published erratum except for Eq.(5) (it was correct in the original paper)',
 'journal-ref': 'J.Quant.Spectrosc.Radiat.Transf. 106, 558-589 (2007); Erratum:\n  J.Quant.Spectrosc.Radiat.Transf. 171, 82-83 (2016)',
 'doi': '10.1016/j.jqsrt.2007.01.034 10.1016/j.jqsrt.2015.11.025',
 'report-no': None,
 'categories': 'physics.optics physics.comp-ph',
 'license': 'http://creativecommons.org/licenses/by-nc-nd/4.0/',
 'abstract': '  We present a review of the discrete dipole approximation (DDA), which is a\ngeneral method to simulate light scattering by arbitrarily shaped particles. We\nput the method in historical context and discuss recent developments, taking\nthe viewpoint of a general framework based on the integral equation

Тестовый датасет:

In [7]:
test_sample_df = pd.read_csv(TEST_SAMPLE_PATH)

In [8]:
test_sample_df.head()

,id,abstract,query
0,2412.16732,A new platinate was recently discovered when...,What unique composition and decomposition beha...
1,nucl-th/9602019,The production cross sections of various fra...,How does the inclusion of statistical decay af...
2,2501.05500,This survey provides a comprehensive examina...,What are the core components of modern zero-kn...
3,2506.20892,A critical challenge for operating fusion burn...,How does impurity seeding affect the timing an...
4,2208.02031,"In this work, we present the first corpus fo...",What is the primary challenge of the newly dev...


In [9]:
test_sample_df.iloc[0]['query']

'What unique composition and decomposition behavior does the newly discovered platinate Nd10.67Pt4O24 exhibit under thermal conditions?'

In [10]:
test_sample_df.iloc[0]['abstract']

"  A new platinate was recently discovered when Nd2O3 was explored as a platinum\ncapture material in the Ostwald process, formed by a direct reaction between\nPtO2(g) and Nd2O3. The crystal structure of this new platinate and its\ncomposition, Nd10.67Pt4O24 , are here reported for the first time. The compound\nis synthesized either by a direct reaction using PtO2(g) or by the citric acid\nchemical route. Based on 3-dimensional electron diffraction data and Rietveld\nrefinement of high-resolution synchrotron and neutron powder diffraction data,\nwe accurately describe its crystal structure in space group I41/a. The compound\nis structurally related to the Ln11-xSrxIr4O24 (Ln = La, Pr, Nd, Sm) phases\nwith a double-perovskite (A2BB'O6) like crystal structure and A-site\ncation-deficiency. Owing to the fixed oxidation state of Pt(IV), two of the\nfour Nd-sites are partly occupied to provide charge neutrality, with the Nd4\nsite taking a split position. On heating, Nd10.67Pt4O24 decompose

In [11]:
test_sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1000 non-null   object
 1   abstract  1000 non-null   object
 2   query     1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


Посмотрим, все ли тестовые данные представлены в RAG-датасете:

In [12]:
arxiv_metadata_df = json_normalize(arxiv_metadata)

In [13]:
arxiv_metadata_df.head(2)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0038,Maxim A. Yurkin,"Maxim A. Yurkin, Alfons G. Hoekstra",The discrete dipole approximation: an overview...,"36 pages, 1 figure; added several corrections ...","J.Quant.Spectrosc.Radiat.Transf. 106, 558-589 ...",10.1016/j.jqsrt.2007.01.034 10.1016/j.jqsrt.20...,None,physics.optics physics.comp-ph,http://creativecommons.org/licenses/by-nc-nd/4.0/,We present a review of the discrete dipole a...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2022-03-30,"[[Yurkin, Maxim A., ], [Hoekstra, Alfons G., ]]"
1,0704.0057,Philipp Werner,Philipp Werner and Andrew J. Millis,High-spin to low-spin and orbital polarization...,Published version,"Phys. Rev. Lett. 99, 126405 (2007)",10.1103/PhysRevLett.99.126405,None,cond-mat.str-el,None,We study the interplay of crystal field spli...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2009-11-13,"[[Werner, Philipp, ], [Millis, Andrew J., ]]"


In [14]:
merged_df = pd.merge (test_sample_df, arxiv_metadata_df, how='inner', on='id')
merged_df.shape[0]

1000

In [15]:
merged_df.head(1)

,id,abstract_x,query,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract_y,versions,update_date,authors_parsed
0,2412.16732,A new platinate was recently discovered when...,What unique composition and decomposition beha...,{\O}ystein Slagtern Fjellv{\aa}g,"{\O}ystein Slagtern Fjellv{\aa}g, Helmer Fjell...","Crystal structure of Nd10.67Pt4O24, a new neod...",None,None,None,None,cond-mat.mtrl-sci,http://creativecommons.org/licenses/by/4.0/,A new platinate was recently discovered when...,"[{'version': 'v1', 'created': 'Sat, 21 Dec 202...",2024-12-24,"[[Fjellvåg, Øystein Slagtern, ], [Fjellvåg, He..."


Да, все тестовые образцы нашлись в RAG-датасете, это хорошо.

In [16]:
merged_df[merged_df['abstract_x'] == merged_df['abstract_y']].shape[0]

1000

И их абстракты совпадают, что тоже хорошо, т. к. не создаёт неоднозначностей сравнения.

## Выводы по итогам EDA

- датасет для RAG
    - 98213 статей с arxiv
    - статьи доступны для скачивания с arxiv
- test_sample.csv
    - 1000 тестовых примеров
    - все тестовые образцы нашлись в RAG-датасете, их абстракты совпадают.
- задачу планируется решать так:
    - Построим эмбеддинги для всех абстрактов RAG-датасета, добавим их в KB вместе с текстами абстрактов. Будем использовать для этого `Qwen/Qwen3-Embedding-0.6B`.
    - применим также ранжирующую модель `Qwen/Qwen3-Reranker-0.6B`

# Реализация retrieval-системы

In [17]:
!pip install langchain -q

In [18]:
!pip install langchain_community langchain_core langchain_text_splitters -q

In [19]:
!pip install jq -q

In [20]:
class RAG:

    def __init__(
        self,
        embedder_name: str = "Qwen/Qwen3-Embedding-0.6B",
        reranker_name: str = "Qwen/Qwen3-Reranker-0.6B",
        chunk_size: int = 500,
        chunk_overlap: int = 125,
        device: Optional[str] = None,
    ):
        self.device = device or ("cuda"
                                 if torch.cuda.is_available() else "cpu")
        self.emb_tokenizer = AutoTokenizer.from_pretrained(embedder_name)
        self.embedder = AutoModel.from_pretrained(embedder_name).to(
            self.device)
        self.embedder.eval()

        self.rr_tokenizer = AutoTokenizer.from_pretrained(
            reranker_name,
            padding_side='left')
        self.reranker = AutoModelForCausalLM.from_pretrained(
            reranker_name).to(self.device)
        self.reranker.eval()

        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
        )
        self.index = None
        self.doc_store = []

        self.max_length = 8192
        self.token_false_id = self.rr_tokenizer.convert_tokens_to_ids("no")
        self.token_true_id = self.rr_tokenizer.convert_tokens_to_ids("yes")
        prefix = "<|im_start|>system\nJudge whether the Document meets the requirements based on the Query and the Instruct provided. Note that the answer can only be \"yes\" or \"no\".<|im_end|>\n<|im_start|>user\n"
        suffix = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"
        self.prefix_tokens = self.rr_tokenizer.encode(prefix,
                                                      add_special_tokens=False)
        self.suffix_tokens = self.rr_tokenizer.encode(suffix,
                                                      add_special_tokens=False)

    def _generate_embeddings(self, texts: List[str]) -> np.ndarray:
        inputs = self.emb_tokenizer(
            texts,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=self.max_length,
        ).to(self.device)

        with torch.no_grad():
            outputs = self.embedder(**inputs)

        inputs.to("cpu")
        embeddings = self.last_token_pool(outputs.last_hidden_state,
                                          inputs.attention_mask).cpu()
        return F.normalize(embeddings, p=2, dim=1).numpy()

    @staticmethod
    def last_token_pool(last_hidden_states: Tensor,
                        attention_mask: Tensor) -> Tensor:
        left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
        if left_padding:
            return last_hidden_states[:, -1]
        else:
            sequence_lengths = attention_mask.sum(dim=1) - 1
            batch_size = last_hidden_states.shape[0]
            return last_hidden_states[
                torch.arange(batch_size, device=last_hidden_states.device),
                sequence_lengths]

    def load_and_process_file(self, file_path: str) -> List[Document]:
        """Загрузка и экстракция данных из файлов"""
        loader = JSONLoader(
            file_path,
            jq_schema=".[] | {id: .id, abstract: .abstract}",
            text_content=False
        )
        docs = loader.load()
        if JUST_DEBUG_WO_GPU:
            docs = docs[:DEBUG_MAX_ELEMENTS]
        return self.text_splitter.split_documents(docs)

    def build_index(self, file_paths: List[str], batch_size: int = 32) -> None:
        """Строим индекс FAISS"""
        all_docs = []
        for path in file_paths:
            all_docs.extend(self.load_and_process_file(path))
        self.doc_store = all_docs

        # Вычислим numpy-эмбеддинги по батчам
        embeddings = []
        for i in range(0, len(all_docs), batch_size):
            batch = [doc.page_content for doc in all_docs[i:i + batch_size]]
            embeddings.append(self._generate_embeddings(batch))

        embeddings = np.concatenate(embeddings)

        # Инициализируем индекс
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(embeddings)

    @staticmethod
    def get_detailed_instruct(task_description: str, query: str):
        return f'Instruct: {task_description}\nQuery:{query}'

    @staticmethod
    def format_reranker_instruction(query, doc, instruction=None):
        if instruction is None:
            instruction = 'Given a web search query, retrieve relevant passages that answer the query'
        output = "<Instruct>: {instruction}\n<Query>: {query}\n<Document>: {doc}".format(
            instruction=instruction, query=query, doc=doc)
        return output

    def process_inputs(self, pairs):
        """Обработка данных для реранкера"""
        inputs = self.rr_tokenizer(pairs,
                                   padding=False,
                                   truncation='longest_first',
                                   return_attention_mask=False,
                                   max_length=self.max_length -
                                   len(self.prefix_tokens) -
                                   len(self.suffix_tokens))
        for i, ele in enumerate(inputs['input_ids']):
            inputs['input_ids'][
                i] = self.prefix_tokens + ele + self.suffix_tokens
        inputs = self.rr_tokenizer.pad(inputs,
                                       padding=True,
                                       return_tensors="pt",
                                       max_length=self.max_length)

        # переносим тензоры на девайс ранжирующей модели
        for key in inputs:
            inputs[key] = inputs[key].to(self.device)
        return inputs

    def search(self,
               query: str,
               k: int = 5,
               task: str = None):
        if self.index is None:
            raise ValueError("Index not initialized")

        if task is None:
            task = 'Given a web search query, retrieve relevant passages that answer the query'

        query_embedding = self._generate_embeddings([query])
        distances, indices = self.index.search(query_embedding, k)
        return distances, indices

    @torch.no_grad()
    def compute_logits(self, inputs):
        batch_scores = self.reranker(**inputs).logits[:, -1, :]
        true_vector = batch_scores[:, self.token_true_id]
        false_vector = batch_scores[:, self.token_false_id]
        batch_scores = torch.stack([false_vector, true_vector], dim=1)
        batch_scores = torch.nn.functional.log_softmax(batch_scores, dim=1)
        scores = batch_scores[:, 1].exp().tolist()
        return scores

    def rerank(self, query: str, documents: List[str], batch_size=1):
        pairs = []
        for d in documents:
            pairs.append(self.format_reranker_instruction(query, d))

        scores = []
        for i in range(0, len(pairs), batch_size):
            inputs = self.process_inputs(pairs[i:i + batch_size])
            sc = self.compute_logits(inputs)
            scores.extend(sc)
        return scores
    
    
    def get_MRR_by_QnA (self, question: str, correct_answer: str) -> str:
        # вопрос к RAG
        D, I = self.search(question, k=100)
        candidates = [self.doc_store[i].page_content for i in I[0]]
        scores = self.rerank(question, candidates)

        k = 5
        array = np.array(scores)
        indices = np.argsort(array)[::-1][:k]

        for i, v in enumerate(indices):
            if candidates[v] == correct_answer:
                return 1.0 / float(i+1) # MRR
        
        return .0

In [ ]:
%%time
rag = RAG()

CPU times: user 4.9 s, sys: 2.71 s, total: 7.61 s
Wall time: 18.9 s


In [22]:
%%time
rag.build_index([ARXIV_METADATA_PATH])

CPU times: user 3h 3min 19s, sys: 32min 9s, total: 3h 35min 28s
Wall time: 3h 33min 46s


# Оценка качества системы

In [23]:
if JUST_DEBUG_WO_GPU:
    test_sample_df = test_sample_df.head(DEBUG_MAX_ELEMENTS)

In [24]:
test_sample_df.shape

(1000, 3)

In [25]:
%%time

def get_MRR (row):
    return rag.get_MRR_by_QnA (row['query'], row['abstract'])
    
test_sample_df ['MRR'] = test_sample_df.apply (get_MRR, axis=1)

avg_MRR = test_sample_df ['MRR'].mean()

print (f"Среднее MRR по всем вопросам = {avg_MRR:.2f}")

You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ubuntu/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2714: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Среднее MRR по всем вопросам = 0.00
CPU times: user 2h 22min 8s, sys: 12.4 s, total: 2h 22min 20s
Wall time: 2h 22min 19s


# Выводы

1. Судя по времени индексации и получившемуся avg_MRR = 0, я что-то делаю не так - прошу ревьюера направить на верный путь.
2. Что касается запрошенных в задании замеров времени - аналогично.